### APIs provided by Reddit will be used to scrape data from Reddit 
### Beautiful Soup and Selinium will be used to scrape data from Hardwarezone since no APIs are provided

# Scraping data from the /r/Singapore subreddit #

In [65]:
import json
import praw
import pandas as pd
import re

In [66]:
with open("credentials.json") as f:
    params = json.load(f)
reddit = praw.Reddit(client_id=params['client_id'], 
                     client_secret=params['api_key'],
                     password=params['password'], 
                     user_agent='<lsj_tester> accessAPI:v0.0.1 (by /u/<yourusername>)',
                     username=params['username'])

In [107]:
coronav = []
for submission in reddit.subreddit('singapore').search('coronavirus', sort='new', limit=10000):
    coronav.append(submission)

In [109]:
cv19 = []
for submission in reddit.subreddit('singapore').search('covid-19', sort='new', limit = 10000):
    cv19.append(submission)

In [111]:
final_list = coronav
for i in cv19:
    if i not in coronav:
        final_list.append(i)

In [115]:
def get_comments(submissions):
    l = []
    for i in submissions:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            if comment.body != "[deleted]" and comment.body!="[removed]":
                l.append(comment.body)
    df = pd.DataFrame(l)
    return df

In [116]:
df = get_comments(final_list)

In [117]:
df.to_csv("reddit.csv")

In [367]:
df = pd.read_csv("reddit.csv")

In [368]:
df.shape

(16345, 2)

# Scraping from HardwarezoneSG #

In [2]:
# codes adapted from user freedaemons on github
# https://github.com/freedaemons/hwz-scrape

In [345]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from datetime import datetime
import time 

In [2]:
pd.set_option('display.max_colwidth', -1)

In [4]:
hwz_url = "https://www.hardwarezone.com.sg/search/forum/covid-19"

In [218]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

#### Extracting the relevant HWZ thread URLs for scraping later ####

In [236]:
driver = webdriver.Firefox()

In [237]:
driver.implicitly_wait(30)
driver.get(hwz_url)
l = []

In [313]:
driver.execute_script("window.scrollTo(0, 1200)") 
nextpage = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/div/div/div/section[2]/section/div[1]/div/div[1]/div/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[{}]'.format(10))
nextpage.click()

In [320]:
soup_level2=BeautifulSoup(driver.page_source, 'lxml')
x = soup_level2.find_all("div", {"class": "gsc-webResult gsc-result"})

In [324]:
for p in x:
    s = p.a["data-ctorig"]
    n = s.rfind("-")
    if n >= len(s) - 9:
        print(s[:n] + ".html")
        s = s[:n] + ".html"
        if s not in l:
            l.append(s)
            continue
    print(s)
    if s not in l:
        l.append(s)

https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-only-beginning-covid-19-global-pandemic-6213702.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-only-beginning-covid-19-global-pandemic-6213702.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/amazing-covid-19-antibodies-csi-now-allows-moh-find-out-who-likely-infected-you-6216292.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/where-did-covid-19-originate-6228343.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/where-did-covid-19-originate-6228343.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-only-beginning-covid-19-global-pandemic-6213702.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-only-beginning-covid-19-global-pandemic-6213702.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/two-more-cases-discharged%3B-no-new-confirmed-case-covid-19-infection-6214986.html
http

In [335]:
df = pd.DataFrame(l, columns = ["url"])

In [336]:
df.to_csv("HWZ_urls.csv")

In [339]:
df = pd.read_csv("HWZ_urls.csv")
l = df["url"].tolist()

In [366]:
len(l)

73

### Scraping of posts and comments from Hardwarezone ###

In [351]:
site_url = 'https://forums.hardwarezone.com.sg'
def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['thread_url', 'userid', 'timestamp', 'post_text', 'post_number', 'post_id', 'first_quote_post_number'] # 'likes_userid'
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url

    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', text='Next ›') == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', text='Next ›')['href']

        thread_page_posts = thread_page_soup.find('div', {'id': 'posts'})
        
        try: 
            for post in thread_page_posts.find_all('div', {'class': 'post-wrapper'}):
                userid_url = post.find('a', {'class': 'bigusername'})['href']
                userid = ''.join(filter(lambda x: x.isdigit(), userid_url))

                datetime_raw = post.find('a', {'name': lambda x: x and x.find('post') == 0}).nextSibling.strip()
                date_list = datetime_raw.split(',')[0].split('-')
                iso_date = '-'.join(list(reversed(date_list)))
                hour = int(datetime_raw.split(' ')[1][0:2])
                if(datetime_raw.split(' ')[2] == 'PM' and hour < 12):
                    hour += 12
                hour_str = str(hour)
                if(hour < 10):
                    hour_str = '0' + str(hour)
                minute = datetime_raw.split(':')[1][0:2]
                iso_datetime = iso_date + 'T' + hour_str + ':' + minute

                post_text = ""
                try:
                    post_text = post.find('div', {'class': 'post_message'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass

                post_number = int(post.find('a', {'id': lambda x: x and 'postcount' in x, 'target': 'new'}).find('strong').get_text())

                post_id = int(post.find('a', {'id': lambda x: x and 'postcount' in x, 'target': 'new'})['id'].lstrip('postcount'))
                
                first_quote_post_number = np.nan
                try:
                    first_quote = post.find('div', {'class': 'quote'})
                    first_quote_post_number = first_quote.find('span', {'class': 'byline'}).find('a')['href'].split('#')[1].lstrip('post')
                except:
                    pass
                
                row = pd.DataFrame([[thread_url, userid, iso_datetime, post_text, post_number, post_id, first_quote_post_number]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[thread_url, "", "", "", np.nan, np.nan, np.nan]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
    thread_df['post_text'] = thread_df['post_text'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

    return thread_df

In [352]:
def processQueryList(query_list, store=None):
    """Process a subset of the list of queries that needs to be made, storing posts in a dataframe"""
    if store is None:
        store = {}
    for url in query_list:
        #Store list of dataframes of posts from each thread into the dictionary
        store[str(url)] = getPosts(url)
    return store

In [354]:
# use of threading to speed up scraping process 
from threading import Thread

def threadedApiCall(nthreads, master_query_list):
    store = {}
    threads = []
    if(len(master_query_list) < nthreads):
        nthreads = len(master_query_list)
    sublistLength = len(master_query_list)//nthreads
    #split the master query list
    sublists = [master_query_list[x:x+sublistLength] for x in range(0,len(master_query_list), sublistLength)]
    for sublist in sublists:
        t = Thread(target=processQueryList, args=(sublist, store))
        threads.append(t)

    #start the threads
    [t.start() for t in threads]
    #wait for the threads to finish
    [t.join() for t in threads]
    
    return pd.concat(thread_posts for thread_posts in list(store.values()))

In [355]:
forum_pdf = threadedApiCall(10, l)

In [358]:
forum_filepath = os.path.join('data', "hwz" + "-2020-04-12.csv")

In [360]:
forum_pdf.to_csv("hwz_data.csv")

In [364]:
forum_pdf["thread_url"].nunique()

73